In [70]:
import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '9994'
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

In [71]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [72]:
torch.manual_seed(42)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
training_args = TrainingArguments(output_dir='./results', num_train_epochs=12, logging_steps=50, save_strategy=IntervalStrategy.NO,
                                  per_device_train_batch_size=15, per_device_eval_batch_size=15, warmup_steps=50,
                                 weight_decay=0.01, logging_dir='./logs', fp16=True, deepspeed='./ds_config.json')
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m").cuda()
model.resize_token_embeddings(len(tokenizer))
descriptions = pd.read_csv('data.csv')['text']
max_length = max([len(tokenizer.encode(description)) for description in descriptions])
print("Max length: {}".format(max_length))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Max length: 276


In [73]:
class PersonalDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [74]:
tokenizer

GPT2TokenizerFast(name_or_path='EleutherAI/gpt-neo-125m', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<|pad|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [75]:
dataset = PersonalDataset(descriptions, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
Trainer(model=model, args=training_args, train_dataset=train_dataset,
       eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

[2023-06-11 21:37:02,916] [WARNING] [cpu_adam.py:84:__init__] FP16 params for CPUAdam may not work on AMD CPUs
 [WARNING]  cpu_adam cuda is missing or is incompatible with installed torch, only cpu ops can be compiled!
Time to load cpu_adam op: 3.4169466495513916 seconds
Time to load utils op: 0.0009012222290039062 seconds


Using /home/butt_sahab/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
No modifications detected for re-loaded extension module cpu_adam, skipping build step...
Loading extension module cpu_adam...
Using /home/butt_sahab/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
No modifications detected for re-loaded extension module utils, skipping build step...
Loading extension module utils...


Rank: 0 partition count [1] and sizes[(125200128, False)] 


Using /home/butt_sahab/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
No modifications detected for re-loaded extension module utils, skipping build step...
Loading extension module utils...


Time to load utils op: 0.0033655166625976562 seconds


Step,Training Loss
50,2.500100
100,0.265500
150,0.175300
200,0.148100
250,0.130300
300,0.113900
350,0.095800
400,0.082100
450,0.068800
500,0.058900


TrainOutput(global_step=720, training_loss=0.26534559461805557, metrics={'train_runtime': 1690.7174, 'train_samples_per_second': 6.281, 'train_steps_per_second': 0.426, 'total_flos': 1495369800220672.0, 'train_loss': 0.26534559461805557, 'epoch': 20.0})

In [80]:
generated = tokenizer("<|startoftext|>", return_tensors="pt").input_ids.cuda()

In [121]:
input_text = "<|startoftext|> Muqeeet, one of your closest freinds in the university. He had decided to help you set up a girl and went out with you and her to help smooth it along. But as it turns out he was sitting by her and talking/laughing while excluding you. You are sitting in KFC with muqeet and the girl nameed AX2. Both muqeet and AX2 are sitting together. You: I think we should switch seats right muqeet? Muqeet:"
encoded_input = tokenizer(input_text, return_tensors="pt").input_ids
sample_output=model.generate(encoded_input, 
                do_sample=True, 
                top_k=50,
                max_length=300, 
                top_p=0.5, 
                temperature=0.7,
                num_return_sequences=1)
output_text = tokenizer.decode(sample_output[0],skip_special_tokens=True)
output_text

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


" Muqeeet, one of your closest freinds in the university. He had decided to help you set up a girl and went out with you and her to help smooth it along. But as it turns out he was sitting by her and talking/laughing while excluding you. You are sitting in KFC with muqeet and the girl nameed AX2. Both muqeet and AX2 are sitting together. You: I think we should switch seats right muqeet? Muqeet: No, I'm just trying to keep things in order."

In [78]:
model.save_pretrained("./Flaskapp/uo")

In [79]:
model = AutoModelForCausalLM.from_pretrained("./Flaskapp/uo")